In [1]:
!pip install smolagents plotly geopandas shapely kaleido -q!


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -!


In [5]:
!pip install smolagents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 62.2 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


1-smolagents: Bu, küçük ve hafif yapay zeka ajanları (agents) oluşturmanıza olanak tanır. Yapay zeka ajanları, belirli görevleri yerine getirmek için kullanılan araçlar olabilir. Bu kütüphane, örneğin CodeAgent gibi ajanları yönetmeye ve görevleri çözmeye yardımcı olur.

2-plotly: Plotly, etkileşimli görselleştirmeler oluşturmak için kullanılan güçlü bir Python kütüphanesidir. Veri görselleştirme, grafikler, haritalar ve daha fazlasını yaratmak için yaygın olarak kullanılır.

3-geopandas: Bu, pandas'ı coğrafi verilerle çalışacak şekilde genişleten bir Python kütüphanesidir. GeoPandas, coğrafi verileri (örneğin, harita verileri) okuma, yazma ve işleme işlemleri için kullanılır.

4-shapely: Bu, coğrafi şekillerle çalışmayı kolaylaştıran bir Python kütüphanesidir. GeoPandas ile birlikte kullanılarak geometrik analizler ve hesaplamalar yapılabilir.

5-kaleido: Plotly ile birlikte kullanılabilen bir araçtır ve görselleştirmeleri statik görüntülere dönüştürmek için gereklidir (örneğin, PNG veya PDF gibi).

In [2]:
!pip install litellm -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.4 MB/s eta 0:00:00


In [6]:
import os
import math
import pandas as pd
from typing import Optional, Tuple
from PIL import Image
from smolagents import CodeAgent, GoogleSearchTool, HfApiModel, VisitWebpageTool, tool
from huggingface_hub import login

login('')

# Google Search API anahtarını ayarla
os.environ["SERPER_API_KEY"] = ""  # API anahtarınızı ekleyin

# Hugging Face Model Tanımı
model = HfApiModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")

In [7]:

@tool
def calculate_cargo_travel_time(
    origin_coords: Tuple[float, float],
    destination_coords: Tuple[float, float],
    cruising_speed_kmh: Optional[float] = 750.0,  # Ortalama kargo uçağı hızı
) -> float:
    """
    İki nokta arasındaki kargo uçağı seyahat süresini hesaplar.

    Args:
        origin_coords: Başlangıç noktası (enlem, boylam)
        destination_coords: Varış noktası (enlem, boylam)
        cruising_speed_kmh: Opsiyonel hız km/saat (varsayılan: 750 km/saat)

    Returns:
        float: Tahmini seyahat süresi (saat)
    """

    def to_radians(degrees: float) -> float:
        return degrees * (math.pi / 180)

    # Koordinatları radyana çevir
    lat1, lon1 = map(to_radians, origin_coords)
    lat2, lon2 = map(to_radians, destination_coords)

    # Dünya'nın yarıçapı (km)
    EARTH_RADIUS_KM = 6371.0

    # Haversine formülü ile mesafe hesapla
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.asin(math.sqrt(a))
    distance = EARTH_RADIUS_KM * c

    # %10 ek mesafe payı ve 1 saat ekleme (iniş/kalkış için)
    actual_distance = distance * 1.1
    flight_time = (actual_distance / cruising_speed_kmh) + 1.0

    return round(flight_time, 2)

In [8]:

# Görev Tanımı
task = """Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.
Also give me some supercar factories with the same cargo plane transfer time."""

# Aracı (Agent) oluştur
agent = CodeAgent(
    model=model,
    tools=[GoogleSearchTool("serper"), VisitWebpageTool(), calculate_cargo_travel_time],
    additional_authorized_imports=["pandas"],
    max_steps=20,
)

# Görevi çalıştır ve sonucu al
result = agent.run(task)

# Sonucu yazdır
print("Görev Tamamlandı!")
print(result)


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  batman_locations = web_search(query="Batman filming locations")                                                  
  print(batman_locations)                                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results
0. [The Batman | 2022 - Movie Locations](https://movie-locations.com/movies/b/The-Batman-2022-2.php)

Film locations for The Batman (2022) in Liverpool, London, Glasgow and Chicago.

1. [Where was The Batman filmed? ALL the Filming Locations in 
...](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)

Guide to the all the filming locations of The Batman - The 2022 movie with Robert Pattison filmed in Chicago and 
The UK - The City and the Wayne House.

2. [Batman (1989) - Filming & production - IMDb](https://www.imdb.com/title/tt0096895/locations/)

Filming locations: Knebworth House, Knebworth, Hertfordshire, England, UK (Wayne Manor; exterior) Helpful•21 0 
Acton Lane Power Station, Acton Lane, Acton, ...

3. [12 Batman Movie Locations You Can Visit! - Travel and Leisure 
Asia](https://www.travelandleisureasia.com/global/destinations/batman-movie-locations-you-can-visit/)
Date published: Jan 10, 2023

Most of the filming of Batman movies is done in the Warner Bros studios and across the US, including New York and 
Pittsburgh. You will notice ...

4. [The Dark Knight | 2008 - Movie Locations](https://movie-locations.com/movies/d/Dark-Knight.php)

Discover where The Dark Knight (2008) was filmed around Chicago, as well as in London and Bedfordshire in the UK, 
and briefly in Hong Kong.

5. [Dark Knight Rises Tour: See Batman Movie 
Locations](https://www.travelchannel.com/interests/arts-and-culture/photos/see-batman-movie-locations)

See Batman Movie Locations · Wollaton Hall · Carnegie Mellon University · The Farmiloe Building · Queensboro Bridge
· Wall Street · Mehrangarh Fort · Turda Saline.

6. [Batman Movies and Filming Locations in Los Angeles | LA 
Today](https://www.traveltodayla.com/post/batman-movies-in-la?srsltid=AfmBOopVyz-Ies210NX2vaNl-1T8rpGOrfLya3Og_WXiz
XM4DG4-qeaT)

Westwood, Los Angeles, CA. Step into the world of Batman and Robin with a visit to the Kirkeby Center, the location
of a thrilling scene from the 1966 film ...

7. [The Batman (2022) Locations - Latitude and Longitude 
Finder](https://www.latlong.net/location/the-batman-2022-locations-427)

The Batman (2022) was filmed in Liverpool and London, England, UK. Filming also took place at Cardington Studios 
and Warner Bros. Leavesden Studios.

8. [The Filming Locations of The Batman - 
Reddit](https://www.reddit.com/r/batman/comments/tx71l5/the_filming_locations_of_the_batman/)
Date published: Apr 5, 2022

The Filming Locations of The Batman: r/batman - Royal Liver Building, Liverpool Royal Liver Building, Liverpool 
more

Out: None

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                              
  import re                                                                                                        
                                                                                                                   
  # Gotham coordinates                                                                                             
  gotham_coords = [40.7128, -74.0060]                                                                              
                                                                                                                   
  # Extract Batman filming locations and their coordinates                                                         
  batman_locations = [                                                                                             
      {"name": "Royal Liver Building", "coords": [53.4084, -2.9916]},                                              
      {"name": "Cardington Studios", "coords": [52.0000, 0.0000]},  # Approximate coordinates                      
      {"name": "Warner Bros. Leavesden Studios", "coords": [51.8800, -0.4100]},  # Approximate coordinates         
      {"name": "Knebworth House", "coords": [51.8800, -0.4100]},  # Approximate coordinates                        
      {"name": "Acton Lane Power Station", "coords": [51.5000, -0.2000]},  # Approximate coordinates               
      {"name": "Carnegie Mellon University", "coords": [40.4432, -80.0000]},  # Approximate coordinates            
      {"name": "Wollaton Hall", "coords": [53.4084, -2.9916]},  # Approximate coordinates                          
      {"name": "Queensboro Bridge", "coords": [40.7081, -73.9397]},                                                
      {"name": "Wall Street", "coords": [40.7076, -74.0119]},                                                      
      {"name": "Mehrangarh Fort", "coords": [26.2913, 73.0243]},                                                   
      {"name": "Turda Saline", "coords": [46.2000, 24.1500]},  # Approximate coordinates                           
      {"name": "Kirkeby Center", "coords": [34.0736, -118.4000]},  # Approximate coordinates                       
  ]                                                      

Execution logs:
                          Location          Coordinates  Travel Time (hours)
0             Royal Liver Building   [53.4084, -2.9916]                 8.81
1               Cardington Studios          [52.0, 0.0]                 9.16
2   Warner Bros. Leavesden Studios       [51.88, -0.41]                 9.12
3                  Knebworth House       [51.88, -0.41]                 9.12
4         Acton Lane Power Station         [51.5, -0.2]                 9.16
5       Carnegie Mellon University     [40.4432, -80.0]                 1.74
6                    Wollaton Hall   [53.4084, -2.9916]                 8.81
7                Queensboro Bridge  [40.7081, -73.9397]                 1.01
8                      Wall Street  [40.7076, -74.0119]                 1.00
9                  Mehrangarh Fort   [26.2913, 73.0243]                18.34
10                    Turda Saline        [46.2, 24.15]                11.87
11                  Kirkeby Center    [34.0736, -118.4]                 6.79
12                     Lamborghini   [45.5451, 11.9863]                10.76
13                         Ferrari   [43.8343, 11.2591]                10.82
14                         Porsche    [48.7758, 9.1829]                10.24
15                    Aston Martin   [52.4124, -1.9336]                 8.95
16                          Pagani   [45.8948, 11.3425]                10.66

Out - Final answer:                           Location          Coordinates  Travel Time (hours)
0             Royal Liver Building   [53.4084, -2.9916]                 8.81
1               Cardington Studios          [52.0, 0.0]                 9.16
2   Warner Bros. Leavesden Studios       [51.88, -0.41]                 9.12
3                  Knebworth House       [51.88, -0.41]                 9.12
4         Acton Lane Power Station         [51.5, -0.2]                 9.16
5       Carnegie Mellon University     [40.4432, -80.0]                 1.74
6                    Wollaton Hall   [53.4084, -2.9916]                 8.81
7                Queensboro Bridge  [40.7081, -73.9397]                 1.01
8                      Wall Street  [40.7076, -74.0119]                 1.00
9                  Mehrangarh Fort   [26.2913, 73.0243]                18.34
10                    Turda Saline        [46.2, 24.15]                11.87
11                  Kirkeby Center    [34.0736, -118.4]                 6.79
12                     Lamborghini   [45.5451, 11.9863]                10.76
13                         Ferrari   [43.8343, 11.2591]                10.82
14                         Porsche    [48.7758, 9.1829]                10.24
15                    Aston Martin   [52.4124, -1.9336]                 8.95
16                          Pagani   [45.8948, 11.3425]                10.66

[Step 2: Duration 12.12 seconds| Input tokens: 5,769 | Output tokens: 1,062]

Görev Tamamlandı!
                          Location          Coordinates  Travel Time (hours)
0             Royal Liver Building   [53.4084, -2.9916]                 8.81
1               Cardington Studios          [52.0, 0.0]                 9.16
2   Warner Bros. Leavesden Studios       [51.88, -0.41]                 9.12
3                  Knebworth House       [51.88, -0.41]                 9.12
4         Acton Lane Power Station         [51.5, -0.2]                 9.16
5       Carnegie Mellon University     [40.4432, -80.0]                 1.74
6                    Wollaton Hall   [53.4084, -2.9916]                 8.81
7                Queensboro Bridge  [40.7081, -73.9397]                 1.01
8                      Wall Street  [40.7076, -74.0119]                 1.00
9                  Mehrangarh Fort   [26.2913, 73.0243]                18.34
10                    Turda Saline        [46.2, 24.15]                11.87
11                  Kirkeby Center    [34.0736, -118.4]   

In [9]:
model = HfApiModel(
    "Qwen/Qwen2.5-Coder-32B-Instruct", provider="together", max_tokens=8096

)

web_agent = CodeAgent(
    model=model,
    tools=[
        GoogleSearchTool(provider="serper"),
        VisitWebpageTool(),
        calculate_cargo_travel_time,
    ],
    name="web_agent",
    description="Browses the web to find information",
    verbosity_level=0,
    max_steps=10,
)

In [11]:
from smolagents.utils import encode_image_base64, make_image_url
from smolagents import OpenAIServerModel


def check_reasoning_and_plot(final_answer, agent_memory):
    multimodal_model = OpenAIServerModel("gpt-4o", max_tokens=8096)
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True


manager_agent = CodeAgent(
    model=HfApiModel("deepseek-ai/DeepSeek-R1", provider="together", max_tokens=8096),
    tools=[calculate_cargo_travel_time],
    managed_agents=[web_agent],
    additional_authorized_imports=[
        "geopandas",
        "plotly",
        "shapely",
        "json",
        "pandas",
        "numpy",
    ],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=15,
)

manager_agent.visualize()

CodeAgent | deepseek-ai/DeepSeek-R1
├── ✅ Authorized imports: ['geopandas', 'plotly', 'shapely', 'json', 'pandas', 'numpy']
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name                        ┃ Description                           ┃ Arguments                             ┃
│   ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ calculate_cargo_travel_time │ İki nokta arasındaki kargo uçağı      │ origin_coords (`array`): Başlangıç    │
│   │                             │ seyahat süresini hesaplar.            │ noktası (enlem, boylam)               │
│   │                             │                                       │ destination_coords (`array`): Varış   │
│   │                             │                                       │ noktası (enlem, boylam)               │
│   │                             │                                       │ cruising_speed_kmh (`number`):        │
│   │                             │                                       │ Opsiyonel hız km/saat (varsayılan:    │
│   │                             │                                       │ 750 km/saat)                          │
│   │ final_answer                │ Provides a final answer to the given  │ answer (`any`): The final answer to   │
│   │                             │ problem.                              │ the problem                           │
│   └─────────────────────────────┴───────────────────────────────────────┴───────────────────────────────────────┘
└── 🤖 Managed agents:
    └── web_agent | CodeAgent | Qwen/Qwen2.5-Coder-32B-Instruct
        ├── ✅ Authorized imports: []
        ├── 📝 Description: Browses the web to find information
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name                        ┃ Description                       ┃ Arguments                         ┃
            ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ web_search                  │ Performs a google web search for  │ query (`string`): The search      │
            │                             │ your query then returns a string  │ query to perform.                 │
            │                             │ of the top search results.        │ filter_year (`integer`):          │
            │                             │                                   │ Optionally restrict results to a  │
            │                             │                                   │ certain year                      │
            │ visit_webpage               │ Visits a webpage at the given url │ url (`string`): The url of the    │
            │                             │ and reads its content as a        │ webpage to visit.                 │
            │                             │ markdown string. Use this to      │                                   │
            │                             │ browse webpages.                  │                                   │
            │ calculate_cargo_travel_time │ İki nokta arasındaki kargo uçağı  │ origin_coords (`array`):          │
            │                             │ seyahat süresini hesaplar.        │ Başlangıç noktası (enlem, boylam) │
            │                             │                                   │ destination_coords (`array`):     │
            │                             │                                   │ Varış noktası (enlem, boylam)     │
            │                             │                                   │ cruising_speed_kmh (`number`):    │
            │                             │                                   │ Opsiyonel hız km/saat             │
            │                             │               

In [ ]:
manager_agent.run("""
Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W).
Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in total.
Represent this as spatial map of the world, with the locations represented as scatter points with a color that depends on the travel time, and save it to saved_map.png!

Here's an example of how to plot and return a map:
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,
     color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)
fig.show()
fig.write_image("saved_image.png")
final_answer(fig)

Never try to process strings using code: when you have a string to read, just print it and you'll see it.
""")
manager_agent.python_executor.state["fig"]